# 連続データのFFT処理

時系列データのFFT処理を通じて、numpy,matplotlib,pandasの使い方を復習する。  
FFT処理の手順は下記の通りである  

1. DC成分の除去
2. ローパスフィルタ処理
3. 窓関数
4. FFT

## 準備

1. matplotlibのバックエンドをinlineに設定
2. matplotlib.pyplotのインポート
3. numpyのインポート
4. scipyからsignalモジュールをインポート
5. pandasのインポート

## DC成分の除去
sensor_data.csvをpandasのDataFrameとして読み込む  
データ概要(describe)を確認する

channel_0の平均値が0となるようにデータを加工する  
加工結果をグラフ化する

## ローパスフィルタの設計
サンプリング間隔dtを抽出し、ナイキスト周波数$f_n$,カットオフ周波数$f_c$を決定する。
ただし、fn,fcは下記の通りとする

$$
\begin{align*}
f_n & = \frac{1.0}{2 \cdot dt} \\
f_c & = 0.2 * f_n
\end{align*}
$$

今回カットオフ周波数の-10\[Hz\]を通過周波数$f_{pass}$、+100\[Hz\]を遮断周波数$f_{stop}$とする

下記の通りナイキスト周波数で無次元化した通過周波数$W_{pass}$、遮断周波数$w_{stop}$を定義する  
合わせて、帯域内（$f_{pass}$から$f_c$）の減衰量$R_{pass}$\[dB\]及び、減衰域（$f_c$ から$f_{stop}$）の減衰量$R_{stop}$\[dB\]は下記の通り定義する

$$
\begin{align*}
W_{pass} & = \frac{f_{pass}}{f_n} \\
W_{stop} & = \frac{f_{stop}}{f_n} \\
R_{pass} & = 1 \quad[dB]\\
R_{stop} & = 30 \quad[dB]
\end{align*}
$$

signale.buttordを用いて、無次元化カットオフ周波数$W_n$及び、必要なフィルタ最低次数$N$を計算する  
```python
N, Wn = signal.buttord(Wpass, Wstop, Rpass, Rstop)
```

次に、ローパスのIIRフィルタを表す多項式の分子b,分母aを計算する  
```python
b, a = signal.butter(N, Wn, "low")
```

さらにa,bであらわされるフィルタをchannel_0のデータに適用する
```python
y_low = signal.filtfilt(b, a, df[df.columns[1]])
```


元のデータ及びローパスをかけた後のデータを重ねてプロットする

尚、適用したフィルタの特性はアナログフィルタとして確認する

```python
b_ana, a_ana = signal.butter(N, Wn, "low", analog=True)
w, h = signal.freqs(b_ana, a_ana)
plt.plot(w, 20*np.log10(abs(h)))
```

b_ana,a_anaはアナログフィルタとしての定数である。また、wは無次元周波数である。
合わせて、カットオフ周波数(無次元)の表示する  
あるxの値に縦線を引きたい場合、下記のようにする。
```python
plt.axvline(Wn, color='green',label="cutoff freq")
```

WstopやWpassもプロットし、フィルタが設計通りの特性になっていることを確認する

## 窓関数
```np.hamming(size)```を用いることで容易にハミング窓が作成できる。  
これをローパスフィルタ適用済みのy_lowと掛け合わせる

プロットすることで、初期時間と最終時間の振幅が小さくなっていることがわかる

## FFT処理

下記のようにfftfreqメソッドを用いれば、周波数配列を作成できる。
ただし、データは前半分でよい（後ろ半分は不要）  

```python
np.fft.fftfreq(y_win.size,dt)
```

さらにfftメソッドを用いればFFTできる    
こちらもデータは前半分でよい
```python
np.fft.fft(y_data)
```

元データ、ローパスフィルタをかけた後のデータ、窓関数も適用したデータの３データ分FFTする

元データ、ローパスフィルタをかけた後のデータ、窓関数も適用したデータのFFTの振幅をプロットする。  
ローパスフィルタにて90\[Hz\]付近から減衰していることがわかる。   
また、窓関数の減衰の影響も確認できる  

## おまけ
上凸のピーク値検出には、signal.argrelmaxが便利である
該当のindexを取得できる。
```python
maxp = signal.argrelmax(np.abs(y_fft_win),order=30)
```

```np.abs(y_fft_win)```のplot以外に、```plt.scatter(freq[maxp],np.abs(y_fft_win[maxp]))```を行うといい感じに表示される

## 参考文書

[実践に即したデジタル フィルター設計の紹介](https://jp.mathworks.com/help/signal/examples/practical-introduction-to-digital-filter-design.html) MathWorks


---

<div align="right">This Text by Katayama is licensed under <a harf="https://creativecommons.org/licenses/by/4.0/">a Creative Commons Attribution 4.0 International License. </a>   </div>
<img src="./by.png" width=100pt align="right">
